# 1. Install and Import Baseline Dependecies

In [10]:
pip install transformers


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
pip install bs4


SyntaxError: invalid syntax (3307789763.py, line 1)

In [14]:
pip install torch


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install ipywidgets

     -------------------------------------- 134.4/134.4 kB 2.0 MB/s eta 0:00:00
     -------------------------------------- 384.1/384.1 kB 8.0 MB/s eta 0:00:00
     ---------------------------------------- 2.0/2.0 MB 16.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from bs4 import BeautifulSoup
import requests
from transformers import PegasusTokenizer, PegasusForConditionalGeneration

# 2. Setup Summarization Model

In [5]:
pip install sentencepiece

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
model_name = "human-centered-summarization/financial-summarization-pegasus"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

Downloading: 100%|████████████████████████████████████████████████████████████████| 2.28G/2.28G [00:48<00:00, 47.3MB/s]


# 3. Summarize a Single Article

In [12]:
article_link = 'https://finance.yahoo.com/news/collapsed-ftx-owes-nearly-3-090734717.html'

In [13]:
r = requests.get(article_link)

In [14]:
soup = BeautifulSoup(r.text, 'html.parser')

In [15]:
paragraphs =soup.find_all('p')

In [16]:
paragraphs

[<p>(Reuters) - Cryptocurrency exchange FTX, which has filed for U.S. bankruptcy court protection, said it owes its 50 biggest creditors nearly $3.1 billion.</p>,
 <p>The exchange owes about $1.45 billion to its top ten creditors, it said in a court filing on Saturday, without naming them.</p>,
 <p>FTX and its affiliates filed for bankruptcy in Delaware on Nov. 11 in one of the highest-profile crypto blowups, leaving an estimated 1 million customers and other investors facing total losses in the billions of dollars.</p>,
 <p>The crypto exchange said on Saturday it has launched a strategic review of its global assets and is preparing for the sale or reorganization of some businesses.</p>,
 <p>(Reporting by Juby Babu in Bengaluru; Editing by Angus MacSwan)</p>]

In [17]:
text = [paragraph.text for paragraph in paragraphs]

In [19]:
text

['(Reuters) - Cryptocurrency exchange FTX, which has filed for U.S. bankruptcy court protection, said it owes its 50 biggest creditors nearly $3.1 billion.',
 'The exchange owes about $1.45 billion to its top ten creditors, it said in a court filing on Saturday, without naming them.',
 'FTX and its affiliates filed for bankruptcy in Delaware on Nov. 11 in one of the highest-profile crypto blowups, leaving an estimated 1 million customers and other investors facing total losses in the billions of dollars.',
 'The crypto exchange said on Saturday it has launched a strategic review of its global assets and is preparing for the sale or reorganization of some businesses.',
 '(Reporting by Juby Babu in Bengaluru; Editing by Angus MacSwan)']

In [24]:
words = ' '.join(text).split(' ')[:200]

In [26]:
Article  = ' '.join(words)

In [27]:
Article

'(Reuters) - Cryptocurrency exchange FTX, which has filed for U.S. bankruptcy court protection, said it owes its 50 biggest creditors nearly $3.1 billion. The exchange owes about $1.45 billion to its top ten creditors, it said in a court filing on Saturday, without naming them. FTX and its affiliates filed for bankruptcy in Delaware on Nov. 11 in one of the highest-profile crypto blowups, leaving an estimated 1 million customers and other investors facing total losses in the billions of dollars. The crypto exchange said on Saturday it has launched a strategic review of its global assets and is preparing for the sale or reorganization of some businesses. (Reporting by Juby Babu in Bengaluru; Editing by Angus MacSwan)'

In [29]:
input_ids = tokenizer.encode(Article, return_tensors='pt')

In [30]:
input_ids

tensor([[  143, 25445,   158,   233, 52399,  2264,  1091, 19227,   108,   162,
           148,  3252,   118,   475,   107,   283,   107,  7073,  1462,  1471,
           108,   243,   126, 26708,   203,  1149,  1715, 13433,  1517, 79924,
          1722,   107,   139,  2264, 26708,   160, 41205,  1343,  1722,   112,
           203,   349,  2069, 13433,   108,   126,   243,   115,   114,  1462,
          5809,   124,  1327,   108,   347, 14537,   183,   107,  1091, 19227,
           111,   203, 11834,  3252,   118,  7073,   115, 11005,   124,  4747,
           107,  1073,   115,   156,   113,   109,  1330,   121, 12651, 11625,
          6049,  5151,   108,  2096,   142,  3627,   305,   604,   527,   111,
           176,  2714,  2931,   916,  5054,   115,   109, 13353,   113,  2729,
           107,   139, 11625,  2264,   243,   124,  1327,   126,   148,  2365,
           114,  3112,   933,   113,   203,  1122,  2718,   111,   117,  3946,
           118,   109,   925,   132, 38771,   113,  

In [32]:
output = model.generate(input_ids, max_length=55, num_beams = 5, early_stopping=True)

In [33]:
summary = tokenizer.decode(output[0] , skip_special_tokens=True)

In [34]:
summary

'FTX says it owes about $1.45 billion to its top ten creditors. Exchange filed for bankruptcy on Nov. 11 in one of the highest-profile blowups'

# 3. Building a News and Sentiment Pipeline

In [36]:
monitored = ['TSL', 'ETH', 'BTC', 'META' , 'AAPL']

## 3.1 Search for stock news using  Yahoo Finance

In [61]:
def search_news_urls(ticker):
    url = f"https://news.search.yahoo.com/search;_ylt=AwrFD22hPHpjY78wWg9XNyoA;_ylu=Y29sbwNiZjEEcG9zAzEEdnRpZAMEc2VjA3BpdnM-?p={ticker}&fr2=piv-web&fr=news"
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    atags = soup.find_all('a', attrs={'class':'thmb'})
    links = [link['href'] for link in atags]
    return links
             

In [62]:
search_news_urls('BTC')

['https://cointelegraph.com/news/gbtc-bitcoin-discount-nears-50-on-ftx-woes-as-investors-stock-up',
 'https://www.fool.com/investing/2022/11/20/biggest-crypto-investors-are-dollar-cost-averaging/',
 'https://www.forbes.com/sites/billybambrough/2022/11/20/in-huge-trouble-10-billion-earthquake-could-be-about-to-cause-bitcoin-and-ethereum-price-chaos/',
 'https://www.fool.com/investing/2022/11/15/should-you-buy-bitcoin-while-its-still-below-20000/',
 'https://seekingalpha.com/article/4558726-bitcoin-could-drop-below-10k-crypto-winter',
 'https://ca.investing.com/news/cryptocurrency-news/btc-still-has-bearish-momentum-to-overcome-price-levels-to-watch-2835359',
 'https://ca.investing.com/news/cryptocurrency-news/btcs-current-position-in-its-market-cycle-is-a-decisive-point-2835767',
 'https://ca.investing.com/news/cryptocurrency-news/can-the-bulls-push-btc-price-to-17000-in-a-lethargic-backdrop-2835875',
 'https://cointelegraph.com/news/bitcoin-sees-record-stock-to-flow-miss-btc-price-mode